# Hello, PyTorchSim!

In [1]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)

## One Touch Simulation
### Normal Matmul Code

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(0)
input = torch.randn(128, 128).to(device)
weight = torch.randn(128, 128).to(device)

opt_fn = torch.compile(torch.matmul)
cpu_out = opt_fn(input, weight)

### PyTorchSim Matmul Code

In [3]:
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

torch.manual_seed(0)
input = torch.randn(128, 128).to(device)
weight = torch.randn(128, 128).to(device)

opt_fn = torch.compile(torch.matmul)
npu_out = opt_fn(input, weight)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/npu/build.ninja...
Building extension module npu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module npu...


ninja: no work to do.
Wrapper Codegen Path = /tmp/torchinductor_root/ro/croutbd6yxrzgdstfcplx7yrpn2do5frwhyx2md5r7rvrubdhdgd.py
[Gem5] Gem5 is running... 
[Spike] Running Spike simulator
[TOGSim] TOGSim is running..  
[TOGSim] Simulation of "/tmp/torchinductor/tmp/fy6nnyudtno/tile_graph.onnx" is stored to "/tmp/torchinductor/tmp/fy6nnyudtno/togsim_result/0"


In [4]:
def test_result(name, npu_out, cpu_out, rtol=1e-4, atol=1e-4):
    if torch.allclose(npu_out.cpu(), cpu_out, rtol=rtol, atol=atol):
        message = f"|{name} Test Passed|"
        print("-" * len(message))
        print(message)
        print("-" * len(message))
    else:
        message = f"|{name} Test Failed|"
        print("-" * len(message))
        print(message)
        print("-" * len(message))
        print("npu out: ", npu_out.cpu())
        print("cpu out: ", cpu_out)
        exit(1)

In [5]:
test_result("MatMul", npu_out, cpu_out)

--------------------
|MatMul Test Passed|
--------------------


### Single kernel mode (TODO: remove it?)

In [2]:
import torch
from torchvision.models import resnet18
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

input = torch.randn(1, 3, 224, 224).to(device=device)
model = resnet18().to(device=device)

opt_fn = torch.compile(dynamic=False)(model)
npu_out = opt_fn(input)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
No modifications detected for re-loaded extension module npu, skipping build step...
Loading extension module npu...


KeyboardInterrupt: 